# Predicting Droughts with Meteorological Data


In [1]:
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm.auto import tqdm
from datetime import datetime
from scipy.interpolate import interp1d
from sklearn.preprocessing import RobustScaler

## Load our saved Data
We can index to get submatrices for less previous observations

In [14]:
def np_load(filename):
    path = ''
    return np.load(path + filename + '.npy', allow_pickle=True)

X_train = np_load('X_train')
y_train = np_load('y_train')
X_valid = np_load('X_valid')
y_valid = np_load('y_valid')

In [12]:
df = pd.read_csv('train_timeseries.csv')

In [20]:
l = list(df.columns)

In [21]:
df[l[:5] + ['score']].head()

,fips,date,PRECTOT,PS,QV2M,score
0,1001,2019-01-01,2.25,100.51,9.69,0.0
1,1001,2019-01-02,4.94,100.48,8.65,NaN
2,1001,2019-01-03,20.74,100.03,8.59,NaN
3,1001,2019-01-04,16.17,99.47,7.93,NaN
4,1001,2019-01-05,0.01,100.39,5.32,NaN


In [17]:
50000 * (180 + 6*6)

10800000

In [15]:
X_train.shape

(103390, 3810)

In [8]:
# convert to multiclass
from viz_report import *

y_train = round_and_intify(y_train)
y_valid = round_and_intify(y_valid)

In [9]:
y_valid.dtype

dtype('int32')

## Ordinal classification with aggregated binary classifiers
https://stackoverflow.com/questions/57561189/multi-class-multi-label-ordinal-classification-with-sklearn


In [30]:
from sklearn.base import BaseEstimator, clone
from multiprocessing import Process, Manager, Pool

class OrdinalClassifier(BaseEstimator):
    def __init__(self, clf, loss, threads=1):
        self.clf = clf
        self.loss = loss #callable
        self.threads = threads
            
    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        
        if self.unique_class.shape[0] > 2:
            if self.threads > 1:
                self._parallel_fit(X, y, self.threads)
            else:
                self._fit(X, y)
        else:
            print('only two label classes')
                
    def _parallel_fit(self, X, y, threads):
        manager = Manager()
        self.clfs = manager.dict()
        #todo: use shared array to save memory
        # for each k - 1 ordinal value we fit a binary classification problem
        
        job = [Process(target=self._parallel_helper, 
                       args=(self.clfs, i, X, y)) 
               for i in range(self.unique_class.shape[0]-1)]
        with Pool(threads) as p:
            _ = [p.start() for p in job]
            _ = [p.join() for p in job]

    def _parallel_helper(self, clfs, i, X, y):
        binary_y = (y > self.unique_class[i]).astype(np.uint8)
        clf = clone(self.clf)
        clf.fit(X, binary_y)
        clfs[i] = clf

    def _fit(self, X, y):
        self.clfs = {}
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf

    def predict_proba(self, X):
        clfs_predict = {k: self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        for i, y in enumerate(self.unique_class):
            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[i][:,1])
            elif i in clfs_predict:
                # Vi = Pr(y > Vi-1) - Pr(y > Vi)
                 predicted.append(clfs_predict[i-1][:,1] - clfs_predict[i][:,1])
            else:
                # Vk = Pr(y > Vk-1)
                predicted.append(clfs_predict[i-1][:,1])
        return np.vstack(predicted).T

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

    def score(self, X, y, sample_weight=None):
        _, indexed_y = np.unique(y, return_inverse=True)
        return self.loss(indexed_y, self.predict(X), sample_weight=sample_weight)

## Experimenting with just a single prediction

In [31]:
w_train = y_train[:, 3]
w_valid = y_valid[:, 3]

In [32]:
base_clf = LogisticRegression()

ord_clf = OrdinalClassifier(base_clf, mean_absolute_error)

In [ ]:
ord_clf.fit(X_train[indices,:], y_train[indices])

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/dkimpara/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/dkimpara/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'OrdinalClassifier' on <module '__main__' (built-in)>


In [10]:
#initialize base classifier
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression

from sklearn.utils.random import sample_without_replacement as sample
from sklearn.ensemble import RandomForestClassifier

base_clf = LogisticRegression()

ord_clf = OrdinalClassifier(base_clf, mean_absolute_error)

In [11]:
from sklearn.multioutput import MultiOutputClassifier as multi
m_clf = multi(ord_clf)

In [12]:
m_clf.get_params()

{'estimator__clf__C': 1.0,
 'estimator__clf__class_weight': None,
 'estimator__clf__dual': False,
 'estimator__clf__fit_intercept': True,
 'estimator__clf__intercept_scaling': 1,
 'estimator__clf__l1_ratio': None,
 'estimator__clf__max_iter': 100,
 'estimator__clf__multi_class': 'auto',
 'estimator__clf__n_jobs': None,
 'estimator__clf__penalty': 'l2',
 'estimator__clf__random_state': None,
 'estimator__clf__solver': 'lbfgs',
 'estimator__clf__tol': 0.0001,
 'estimator__clf__verbose': 0,
 'estimator__clf__warm_start': False,
 'estimator__clf': LogisticRegression(),
 'estimator__loss': <function sklearn.metrics._regression.mean_absolute_error(y_true, y_pred, *, sample_weight=None, multioutput='uniform_average')>,
 'estimator': OrdinalClassifier(clf=LogisticRegression(),
                   loss=<function mean_absolute_error at 0x15f3ff250>),
 'n_jobs': None}

## gridSearch

In [14]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer

def transform_param(grid):
    #utility to rename keys for chained estimator
    prefix = 'estimator__clf__'
    return {prefix + k: grid[k] for k in grid.keys()}

scorer = {'mae': make_scorer(mean_absolute_error),
          'f1': make_scorer(macro_f1)}

In [18]:
from sklearn.model_selection import GridSearchCV

train_num = int(y_train.shape[0] / 10)
indices = sample(y_train.shape[0], train_num)

tic = time.perf_counter()


param_grid = {'C': [1.0,2.0]}

gridSearch = GridSearchCV(m_clf, transform_param(param_grid), 
                          scoring=scorer,
                          n_jobs=1,
                          refit=False,
                          verbose=2,
                          cv=2) 

gridSearch.fit(X_train[indices,:], y_train[indices])

toc = time.perf_counter()
duration = toc - tic
minutes = int(duration/60)
print(f"Elapsed time: {str(minutes) + ' minutes, ' if minutes else ''}{int(duration % 60)} seconds")

Fitting 2 folds for each of 2 candidates, totalling 4 fits


/Users/dkimpara/github/ML class/hw/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


fitted classifier 0
params{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


/Users/dkimpara/github/ML class/hw/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


fitted classifier 1
params{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


...

/Users/dkimpara/github/ML class/hw/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


fitted classifier 2
params{'C': 2.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


/Users/dkimpara/github/ML class/hw/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


fitted classifier 3
params{'C': 2.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


KeyboardInterrupt: 

In [11]:
import joblib
import pandas as pd

def save_model(model, name):
  #save your model or results
  joblib.dump(model, name + '.pkl')

#load your model for further usage


In [9]:
import joblib
ord_clf = joblib.load("ordinal.pkl")



In [11]:
mae(w_valid, ord_clf.predict(X_valid))

0.28191350548369615

## Ordinal Classification, chained/multioutput

In [ ]:
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer

chain = RegressorChain(ord_clf)

In [ ]:
chain.get_params()

In [ ]:
def transform_param(grid):
    #utility to rename keys for chained estimator
    prefix = 'estimator__base_estimator__'
    return {prefix + k: grid[k] for k in d.keys()}

In [ ]:



tic = time.perf_counter()

boostChain = RegressorChain(HistGradientBoostingRegressor())

# can just do inner params because regressorChain doesn't 
#have params we want to grid search over

param_grid = {'loss': ['squared_error', 'absolute_error'],
              'learning_rate': [0.1, 0.5, 1], 
              'max_iter': [25],
              'max_depth': [10,50,70],
              'l2_regularization':[0,7],
              'max_bins': [100,255],
              'scoring': ['loss']}

gridSearch = GridSearchCV(boostChain, transform_param(param_grid)) 
scorer = 
gridSearch.fit(X_train, np.squeeze(y_target_train))

In [ ]:
y_pred_valid = boostChain.predict(X_valid)
summarize(y_target_valid, y_pred_valid)
toc = time.perf_counter()
duration = toc - tic
minutes = int(duration/60)
print(f"Elapsed time: {str(minutes) + ' minutes, ' if minutes else ''}{int(duration % 60)} seconds")